In [ ]:
# default_exp core

In [ ]:
#export
from nbdev.showdoc import *
import importlib
import os
import sys
import inspect
import json
import pkg_resources
import pkgutil
import pandas
import warnings
import pydoc
from pathlib import Path
import distutils.sysconfig as sysconfig

In [ ]:
#export
def replace(list_, str1, str2):
    """Just replacement function"""
    i = list_.index(str1)
    return list_[:i] + [str2] + list_[i + 1:]

In [ ]:
#export
def get_top_n(n):
    """Returns list of n most popular PyPi packages"""
    if not os.path.isfile('data/top.json'):
        os.system('curl -X GET "https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json" > data/top.json')

    f = open('data/top.json')
    data = json.load(f)

    return [dictionary['project'] for dictionary in data['rows'][:n]]

In [ ]:
#export 
def pip_top_n(n):
    """Pips top n packages generated by get_top_hundred"""
    lib_names = get_top_n(n)
    for lib_name in lib_names:
        try:
            os.system('pip install -q ' + lib_name)
            print(lib_name + ' was successfully pipped')
        except Exception:
            pass

In [ ]:
#export
def val(key):
    """Checks if objects's key is valid."""
    return key[0] != '_' and key[-1] != '_'

In [ ]:
#export
def extr(module_name):
    """Recursive descending in module function"""
    texts = []
    paths = []

    finded_texts, finded_paths = recclass(module_name, module_name.__name__)
    texts.extend(finded_texts)
    paths.extend(finded_paths)

    if hasattr(module_name, '__path__'):
        file = module_name.__path__
    else:
        try:
            file = [inspect.getfile(module_name)]
        except TypeError:
            file = [sys.executable]

    for importer, key, ispkg in pkgutil.iter_modules(file):
        if val(key) and ispkg and hasattr(module_name, key):
            if key != module_name.__name__ and fullname(getattr(module_name, key)).startswith(fullname(module_name)):
                finded_texts, finded_paths = extr(getattr(module_name, key))
                texts.extend(finded_texts)
                paths.extend(finded_paths)

    return texts, paths

In [ ]:
#export
def recclass(cl, path):
    """Recursive descending in class function"""
    texts = []
    paths = []

    texts.append(pydoc.render_doc(cl, renderer=pydoc.TextDoc()))
    paths.append(path)

    for key_, value_ in inspect.getmembers(cl, inspect.isroutine):
        if val(key_) and hasattr(cl, key_):
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isfunction):
        if val(key_) and hasattr(cl, key_):
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isdatadescriptor):
        if val(key_) and hasattr(cl, key_):
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isclass):
        if val(key_) and hasattr(cl, key_):
            if fullname(getattr(cl, key_)).startswith(fullname(cl)):
                finded_texts, finded_paths = recclass(getattr(cl, key_), path + '.' + key_)
                texts.extend(finded_texts)
                paths.extend(finded_paths)

    return texts, paths

In [ ]:
#export
def fullname(o):
    """Attempts to return fullname of an object. Works poorly with properties and data descriptors"""

    if inspect.ismodule(o):
        return o.__name__

    if inspect.isfunction(o) or inspect.isclass(o) or inspect.ismethod(o):
        return o.__module__ + '.' + o.__name__

    if inspect.isroutine(o):
        if hasattr(o, '__name__'):
            return o.__class__.__module__ + '.' + o.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__

    if inspect.isdatadescriptor(o):
        if o.__class__.__name__ == 'property' and o.fget != None:
            if o.fget.__module__ != None:
                return o.fget.__module__ + '.' + o.fget.__name__
            else:
                return o.fget.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__

    return o.__class__.__name__

In [ ]:
#export
def is_parent(o2, o1):
    items1 = repr(pydoc.resolve('.'.join(o1.split('.')[:-1]))[0]).split('.')
    items2 = repr(pydoc.resolve('.'.join(o2.split('.')[:-1]))[0]).split('.')
    return items2[-2] == 'base' and items1[:-2] == items2[:-2] and items1[-2] != 'base'

In [ ]:
#export
def is_base(o):
    return 'base' in repr(pydoc.resolve('.'.join(o.split('.')[:-1]))[0]).split('.')

In [ ]:
#export
def cut_names(paths):
    for path in paths:
        if is_base(path):
            paths = [path_ for path_ in paths if not is_parent(path, path_)]
        
    return paths

In [ ]:
#export
def cut(names):
    removes = []
    names = [name.split('.') for name in names]
    for name in names:
        for i in range(len(names)):
            if names[i][-2:] == name[-2:] and len(name) > len(names[i]):
                removes.append(name)
                
    return ['.'.join(name) for name in names if name not in removes]

In [ ]:
#export
def create_df(texts, paths, show_lib=True):
    
    doc = pandas.DataFrame({'text': texts, 'path': paths})
    doc_wo_dupl = doc.drop_duplicates(subset=['path'])

    d = doc_wo_dupl.groupby('text').agg(name = ('path', lambda x: list(x)), #all names
                                        library = ('path', lambda x: list(x)[0].split('.')[0])
                                       ).reset_index()
    
    ind = d[d.library == 'pandas'].index
    
    for i in ind: # pandas-related feature
        d.name[i] = cut(cut_names(d.name[i])) 
        
    d = d.drop(d[d.name.map(len) > 10].index) 
    
    d = d.explode('name').reset_index(drop=True)
    
    d.text = [d.name[ind] + '. ' + d.text[ind][30:] for ind in d.index]
    
    if show_lib == False:
        d = d.drop('library', axis=1)
    
    return d
    

In [ ]:
#export
def extract():
    """Creates dataframe of documentation"""
    warnings.filterwarnings("ignore")
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)


    installed_packages = pkg_resources.working_set
    first_packages_list = sorted(["%s" % i.key for i in installed_packages])
    
    second_packages_list = [name for name in list(sys.modules.keys()) if val(name) and '.' not in name]
    
    installed_packages_list = second_packages_list + list(set(first_packages_list) - set(second_packages_list))
    
    if installed_packages_list.count('transformers'): # due to strange bug
        installed_packages_list.remove('transformers')

    unsolved_package_names = ['beautifulsoup4', 'typing-extensions', 'pyyaml', 'argon2-cffi', 'jupyter-client', 'jupyter-core']
    unsolved_import_names = ['bs4', 'typing', 'yaml', 'argon2', 'jupyter_client', 'jupyter_core']
    
    for i in range(len(unsolved_package_names)): #libs which I can't do in the general way
        if installed_packages_list.count(unsolved_package_names[i]):
            installed_packages_list = replace(installed_packages_list, unsolved_package_names[i], unsolved_import_names[i])

    texts = []
    paths = []
    
    for lib_name in installed_packages_list:
        try:
            importlib.import_module(lib_name)
            
            finded_texts, finded_paths = extr(sys.modules[lib_name])
            texts.extend(finded_texts)
            paths.extend(finded_paths)

        except Exception:
            try:
                metadata_dir = pkg_resources.get_distribution(lib_name).egg_info
                for name in open('%s/%s' % (metadata_dir, 'top_level.txt')).read().rstrip().split('\n'):
                    if name != '' and val(name):
                        importlib.import_module(name)

                        finded_texts, finded_paths = extr(sys.modules[name])
                        texts.extend(finded_texts)
                        paths.extend(finded_paths)

            except Exception:
                print('--------------- exception during ' + lib_name + ' documentation extracting')
    

    return create_df(texts, paths)

In [ ]:
#export
def extract_one(module_name):
    warnings.filterwarnings("ignore")
    
    importlib.import_module(module_name)
    
    texts, paths = extr(sys.modules[module_name])
    
    return create_df(texts, paths, show_lib=False)
    